### Ingestion del archivo "language_role.json"

In [0]:
dbutils.widgets.text('p_enviroment', '')
v_enviroment = dbutils.widgets.get('p_enviroment')

In [0]:
dbutils.widgets.text('p_file_date', '2024-12-16')
v_file_date = dbutils.widgets.get('p_file_date')

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

### Paso 1 - Leer el archivo JSON usando DataFrameReader de Spark

In [0]:
from pyspark.sql.types import *

In [0]:
languages_roles_schema = StructType(fields=[
    StructField('roleId', IntegerType(), True),
    StructField('languageRole', StringType(), True)
])

In [0]:
languages_roles_df = spark.read \
    .schema(languages_roles_schema) \
    .option('multiline', 'true') \
    .json(f'{bronze_folder_path}/{v_file_date}/language_role.json')

### Paso 2 - Renombrar y Agregar Columnas
1. "roleId" renombrar a "role_id"
2. "languageRole" renombrar a "language_role"
3. Agregar columna "ingestion_date"
4. Agregar columna "enviroment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
languages_roles_final_df = add_ingestion_date(languages_roles_df) \
    .withColumnRenamed('roleId', 'role_id') \
    .withColumnRenamed('languageRole', 'language_role') \
    .withColumn('enviroment', lit(v_enviroment)) \
    .withColumn('file_date', lit(v_file_date))

### Escribir la salida en formato "Parquet"

In [0]:
languages_roles_final_df.write.mode('overwrite').format('delta').saveAsTable('movie_silver.languages_roles')

In [0]:
display(spark.read.format("delta").load(f'{silver_folder_path}/languages_roles'))

In [0]:
dbutils.notebook.exit("Success")